In [9]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

#ipython add-ons
from IPython.parallel import Client
from IPython.display import Image
import multiprocessing

##nipype
import nibabel as nib
from nipype.pipeline.engine import Node, MapNode, Workflow
from nipype.interfaces.io import DataGrabber, DataFinder, DataSink
from nipype.interfaces import fsl
from nipype.interfaces.fsl import ImageMeants

%matplotlib inline

In [69]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)
exps = ['sim','ser']
rois = ['wm','ventricles']
runs = map(str,range(1,4))

In [71]:
def extract_roi(in_tuple):
    sub,exp,run,mask = in_tuple
    
    sub_path = home_dir + 'analysis/' + exp + '_8mm/' + sub + '/preproc/run_' + \
    run + '/'

    preproc_data = sub_path + 'unsmoothed_timeseries.nii.gz'
    mask_dir = home_dir + 'data/' + sub + '/masks/' 
    mask_file = mask_dir + mask + '_' + exp  + '.nii.gz'
    out_f = mask_dir + ('').join(map(str,in_tuple))+ '.txt'

    if os.path.exists(sub_path):
        meants = ImageMeants(in_file = preproc_data, eig = True, order = 1, 
                             mask = mask_file, out_file = out_f)
        meants.run()

In [72]:
in_tuples = []
for sub in sub_list:
    for exp in exps:
        for run in runs:
            for mask in rois:
                in_tuples.append((sub,exp,run,mask))  

In [ ]:
pool = multiprocessing.Pool(processes = 25)
pool.map(extract_roi,in_tuples)

In [68]:
for sub in sub_list:
    for exp in exps:
        
        noise_rois = {'wm':[],'ventricles':[],'run':[]}
        out_f = home_dir + 'data/' + sub + '/design/noise_regressors.csv' #out file
        
        for run in runs:
            for n,mask in enumerate(rois):
                
                in_tuple = (sub,exp,run,mask)
                
                ##paths
                sub_path = home_dir + 'analysis/' + exp + '_8mm/' + sub + '/preproc/run_' + \
                run + '/'
                mask_dir = home_dir + 'data/' + sub + '/masks/' 

                #get data
                if os.path.exists(sub_path):
                    data_f = mask_dir + ('').join(map(str,in_tuple))+ '.txt'
                    data = np.loadtxt(data_f)
                    noise_rois[mask].extend(data)
                    if n > 0: #only add run once per iteration through all masks
                        noise_rois['run'].extend([run]*len(data))
        
        noise_rois = pd.DataFrame(noise_rois)
        
        ##write output
        noise_rois.to_csv(out_f, header=True,index = False, columns = ['wm','ventricles','run'])


/data/home/iballard/fd/analysis/sim_8mm/fd_104/preproc/run_1/
236
0.01208508246
/data/home/iballard/fd/analysis/sim_8mm/fd_104/preproc/run_1/
236
1.147413487
/data/home/iballard/fd/analysis/sim_8mm/fd_104/preproc/run_2/
251
-2.601707668
/data/home/iballard/fd/analysis/sim_8mm/fd_104/preproc/run_2/
251
2.597245904
/data/home/iballard/fd/analysis/sim_8mm/fd_104/preproc/run_3/
254
0.03163308941
/data/home/iballard/fd/analysis/sim_8mm/fd_104/preproc/run_3/
254
-0.1862234554
/data/home/iballard/fd/analysis/ser_8mm/fd_104/preproc/run_1/
316
0.13949804
/data/home/iballard/fd/analysis/ser_8mm/fd_104/preproc/run_1/
316
-3.39563464
/data/home/iballard/fd/analysis/ser_8mm/fd_104/preproc/run_2/
317
0.8826098384
/data/home/iballard/fd/analysis/ser_8mm/fd_104/preproc/run_2/
317
1.513571642
/data/home/iballard/fd/analysis/ser_8mm/fd_104/preproc/run_3/
326
-0.1744440809
/data/home/iballard/fd/analysis/ser_8mm/fd_104/preproc/run_3/
326
-0.6714511323
/data/home/iballard/fd/analysis/sim_8mm/fd_105/prepro